# <B> Create Git Repository using CodeCommit </B>
* Container: conda_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 설정

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [4]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

In [5]:
print (f'strBucketName: {strBucketName}')
print (f'strExecutionRole: {strExecutionRole}')

strBucketName: sagemaker-us-east-1-419974056037
strExecutionRole: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436


## 2. CodeCommit 생성하기

In [6]:
import boto3

In [7]:
iam_client = boto3.client('iam')
codecommit_client = boto3.client('codecommit')

### 2.1 CodeCommit 관련 Credentials 생성 및 Parameter store에 저장하기

* CodeCommit Credentials

In [8]:
user_name = 'dongjin' ## ==> IAM에서 사용자 아이디 확인합니다.
codecommit_cred = 'codecommit-cred-' + user_name
print (f'codecommit_cred: {codecommit_cred}')

codecommit_cred: codecommit-cred-dongjin


In [9]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

ServiceUserName : dongjin-at-419974056037 
ServicePassword : NNOrSohFz6EWG7yGDje7FWyGcopbNWexrvyN7sTe8VE=


In [10]:
code_repository_name = 'TRAIN-CODE' ## ==> 사용할 code repository 폴더 명을 넣습니다. ex) model_code
local_code_dir = './sources' ## ==> 생성한 local의 code repository 폴더 명을 넣습니다. ex) code

In [11]:
try:
    response = codecommit_client.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit_client.get_repository(
        repositoryName=code_repository_name
    )

In [12]:
codecommit_repo = response['repositoryMetadata']['cloneUrlHttp']
codecommit_repo

'https://git-codecommit.us-east-1.amazonaws.com/v1/repos/TRAIN-CODE'

In [14]:
!rm -rf .git/

!git init
!git remote add origin $codecommit_repo
!git checkout -b main
!git add $local_code_dir
!git commit -m "code-update"
!git push --set-upstream origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines/.git/
Switched to a new branch 'main'
[main (root-commit) 65af44a] code-update
 15 files changed, 850 insertions(+)
 create mode 100644 sources/evaluation/.ipynb_checkpoints/evaluation-checkpoint.py
 create mode 100644 sources/evaluation/.ipynb_checkpoints/requirements-checkpoint.txt
 create mode 100644 sources/evaluation/evaluation.py
 create mode 100644 sources/evaluation/requirements.txt
 create mode 100644 so

## 3. parameter 저장하기

In [15]:
pm.put_params(key="-".join([strPrefix, "CODECOMMIT-USERNAME"]), value=ServiceUserName, overwrite=True, enc=True)
pm.put_params(key="-".join([strPrefix, "CODECOMMIT-PWD"]), value=ServicePassword, overwrite=True, enc=True)
pm.put_params(key="-".join([strPrefix, "CODE-REPO"]), value=codecommit_repo.replace('https://',''), overwrite=True)  ## https:// 있을 경우 입력 못함

'Store suceess'

In [16]:
print (f'CODE-REPO: {pm.get_params(key="-".join([strPrefix, "CODE-REPO"]), enc=False)}')
print (f'CODECOMMIT-USERNAME: {pm.get_params(key="-".join([strPrefix, "CODECOMMIT-USERNAME"]), enc=False)}')
print (f'CODECOMMIT-PWD: {pm.get_params(key="-".join([strPrefix, "CODECOMMIT-PWD"]), enc=False)}')

CODE-REPO: git-codecommit.us-east-1.amazonaws.com/v1/repos/TRAIN-CODE
CODECOMMIT-USERNAME: AQICAHixC/mZVJcnQEHIgIK/d13m2pRN5MnNJb7dfKPh/9fZ0QFrilThQ9dQDTaj9rVv7WH1AAAAdTBzBgkqhkiG9w0BBwagZjBkAgEAMF8GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQM09Q1uQwVhS25FvOlAgEQgDIzYaRZtjxLyeN/ByJFidEg+38L5QwnbzEXfTLy9RruSVzm6nI4Td1ZnjFVGZllQMqFtQ==
CODECOMMIT-PWD: AQICAHixC/mZVJcnQEHIgIK/d13m2pRN5MnNJb7dfKPh/9fZ0QHH+D/Rg6+ttvaxzS/8qtuTAAAAizCBiAYJKoZIhvcNAQcGoHsweQIBADB0BgkqhkiG9w0BBwEwHgYJYIZIAWUDBAEuMBEEDAgw+IkxgtJ9qspolQIBEIBHA1fZrpEPWkV9khpEAAQMAo45AxxLmN+nlVQU9Z7A1XDRqIxDomG1f2z/isKsKzHQtlV4CnYPrJSWIlheR3pSjlHUtSRTAew=
